In [ ]:
import cv2 as cv
# from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import ipywidgets as widgets

import numpy as np

In [ ]:
img = cv.imread('./lab_01.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
(img_height, img_width, _) = img.shape
img_ratio = 1000 / img_width
img = cv.resize(img, (1000, int(img_height * img_ratio)), interpolation=cv.INTER_AREA)

In [ ]:
plt.imshow(img)

In [ ]:
hsv_img = cv.cvtColor(img, cv.COLOR_RGB2HSV)

In [ ]:
h_img, s_img, v_img = hsv_img[:,:,0], hsv_img[:,:,1], hsv_img[:,:,2]

In [ ]:
plt.imshow(h_img)
plt.colorbar()

In [ ]:
plt.imshow(s_img)
plt.colorbar()

In [ ]:
plt.imshow(v_img)
plt.colorbar()

In [ ]:
slider_lower = widgets.IntSlider(description='Lower', min=0, max=255, step=1)
slider_upper = widgets.IntSlider(description='Upper', min=0, max=255, step=1)
display(widgets.VBox([slider_lower, slider_upper]))

In [ ]:
def masker(mask):
    kernel = np.ones((5,5),np.uint8)
    mask2 = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    mask3 = cv.morphologyEx(mask2, cv.MORPH_CLOSE, kernel)
    masks = np.hstack((mask, mask2, mask3))
    return masks

In [ ]:
def on_slider_change(_):
    mask = cv.inRange(hsv_img, (0, 0, slider_lower.value), (0, 0, slider_upper.value))
    masker(mask)
    #print(slider_lower.value, slider_upper.value)

In [ ]:
slider_lower.observe(on_slider_change, 'value')
slider_upper.observe(on_slider_change, 'value')

In [ ]:
_mask = cv.inRange(
    hsv_img,
    (100, 0, 200),
    (150, 50, 255)
)

In [ ]:
masked = masker(_mask)

In [ ]:
plt.imshow(_mask)

In [ ]:
blur_mask = cv.dilate(_mask, (15, 15), iterations=10)
#blur_mask = cv.blur(blur_mask, (15, 15))
plt.imshow(blur_mask)

In [ ]:
ORIGIN_TOP_LEFT = (0, 0)
ORIGIN_BOTTOM_LEFT = (199, 0)

def roi_corner(mask, area_x, area_y, origin):
    roi_mask = mask[area_x[0]:area_x[1], area_y[0]:area_y[1]]
    roi_mask = np.float32(roi_mask)
    roi_corners = cv.cornerHarris(roi_mask, blockSize=3, ksize=3, k=0.05)
    
    roi_points = np.argwhere(roi_corners > 0.025 * roi_corners.max())

    roi_distances = np.array([np.linalg.norm(origin - p) for p in roi_points])
    roi_index = np.argmin(roi_distances)
    roi_point = roi_points[roi_index]

    return (roi_point[0] + area_x[0], roi_point[1] + area_y[1])

In [ ]:
roi_corner(_mask, (0, 200), (200, 400), ORIGIN_BOTTOM_LEFT)

In [ ]:
top_left_mask = blur_mask[0:200, 200:400]
plt.imshow(top_left_mask)
top_left_mask = np.float32(top_left_mask)
top_left_corners = cv.cornerHarris(top_left_mask, blockSize=3, ksize=3, k=0.05)

In [ ]:
top_left_point = np.argwhere(top_left_corners > 0.025 * top_left_corners.max())[0]

print(top_left_point)

top_left_img = img[0:200, 200:400]
top_left_img[top_left_point[0], top_left_point[1]] = [255, 0, 0]

plt.imshow(top_left_img)

In [ ]:
bottom_left_mask = _mask[400:600, 200:400]
plt.imshow(bottom_left_mask)
bottom_left_mask = np.float32(bottom_left_mask)
bottom_left_corners = cv.cornerHarris(bottom_left_mask, blockSize=3, ksize=3, k=0.05)

In [ ]:
bottom_left_point = np.argwhere(bottom_left_corners > 0.025 * bottom_left_corners.max())

bottom_left_distances = np.array([np.linalg.norm((199, 0) - p) for p in bottom_left_point])
bottom_left_index = np.argmin(bottom_left_distances)
bottom_left_tacka = bottom_left_point[bottom_left_index]

bottom_left_point_swap = np.swapaxes(bottom_left_point, 0, 1)

bottom_left_img = img[400:600, 200:400]
bottom_left_img[bottom_left_tacka[0], bottom_left_tacka[1]] = [255, 0, 0]

plt.imshow(bottom_left_img)

In [ ]:
float_mask = np.float32(blur_mask)
corners = cv.cornerHarris(float_mask, blockSize=3, ksize=3, k=0.05)

In [ ]:
kernel = np.ones((7,7), np.uint8)
corners_dilated = cv.dilate(corners, kernel, iterations= 2)

img[corners_dilated > 0.025 * corners_dilated.max()] = [255, 0, 0]

plt.imshow(img)